<a href="https://colab.research.google.com/github/diegosfc91/Advanced-Machine-Learning/blob/main/Ingenier%C3%ADa_de_Caracter%C3%ADsticas_Creaci%C3%B3n_de_Nuevas_Variables_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Módulo de Carga y Exploración de Datos

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar los datos
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

# Función para realizar una exploración inicial
def explorar_datos(data, nombre_dataset):
    print(f"Exploración inicial del dataset: {nombre_dataset}")
    print("Primeras filas:\n", data.head())
    print("\nInformación del dataset:")
    print(data.info())
    print("\nCantidad de valores nulos:\n", data.isnull().sum())
    print("-" * 40)

# Explorar datos
explorar_datos(train_data, "Train")
explorar_datos(test_data, "Test")


Exploración inicial del dataset: Train
Primeras filas:
    PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  

# 2. Módulo de Limpieza de Datos

In [14]:
# Función para limpiar datos
def limpiar_datos(data):
    # Imputar valores faltantes en 'Age' con la mediana
    data['Age'] = data['Age'].fillna(data['Age'].median())
    # Imputar valores faltantes en 'Fare' con la mediana
    if 'Fare' in data.columns:
        data['Fare'] = data['Fare'].fillna(data['Fare'].median())
    # Eliminar columna 'Cabin' debido a muchos valores faltantes
    if 'Cabin' in data.columns:
        data = data.drop(columns=['Cabin'])
    # Imputar valores faltantes en 'Embarked' con el valor más frecuente
    if 'Embarked' in data.columns:
        data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
    return data

# Limpiar los datos
train_data = limpiar_datos(train_data)
test_data = limpiar_datos(test_data)


# 3. Módulo de Creación de Nuevas Características



In [15]:
# Función para crear nuevas variables
def crear_nuevas_variables(data):
    # Tamaño de la familia: Combinar 'SibSp' y 'Parch'
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1  # +1 para incluir al propio pasajero
    # Cabina conocida: Variable binaria (solo si 'Cabin' existe en los datos)
    if 'Cabin' in data.columns:
        data['CabinKnown'] = data['Cabin'].notnull().astype(int)
    else:
        data['CabinKnown'] = 0  # Si no existe, asume que no hay cabina conocida
    # Categoría de tarifa: Agrupar en rangos
    data['FareCategory'] = pd.cut(data['Fare'], bins=[-1, 7.91, 14.454, 31, 1000],
                                  labels=['Low', 'Mid-Low', 'Mid-High', 'High'])
    # Título del pasajero: Extraer título desde 'Name'
    data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    # Agrupar títulos raros
    rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir',
                   'Jonkheer', 'Dona']
    data['Title'] = data['Title'].replace(rare_titles, 'Rare')
    data['Title'] = data['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
    return data

# Aplicar la función a los datos de entrenamiento y prueba
train_data = crear_nuevas_variables(train_data)
test_data = crear_nuevas_variables(test_data)


# 4. Codificación de Variables Categóricas

In [16]:
# Función para codificar variables categóricas
def codificar_categorias(data):
    # Codificar 'Sex': male -> 0, female -> 1
    data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
    # Codificar 'Embarked': C -> 0, Q -> 1, S -> 2
    if 'Embarked' in data.columns:
        data['Embarked'] = data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
    # Codificar 'FareCategory': Low -> 0, Mid-Low -> 1, etc.
    if 'FareCategory' in data.columns and data['FareCategory'].dtype.name == 'category':
        data['FareCategory'] = data['FareCategory'].cat.codes
    # Asegurar que 'Title' esté codificado numéricamente
    if 'Title' in data.columns and data['Title'].dtype == 'object':
        title_mapping = {'Mr': 1, 'Miss': 2, 'Mrs': 3, 'Master': 4, 'Rare': 5}
        data['Title'] = data['Title'].map(title_mapping)
    return data

# Codificar las variables categóricas
train_data = codificar_categorias(train_data)
test_data = codificar_categorias(test_data)


# 5. Evaluación de Modelos


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score

# Dividir los datos para modelos con y sin nuevas características
base_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
new_features = base_features + ['FamilySize', 'CabinKnown', 'FareCategory', 'Title']

# Sin nuevas características
X_base = train_data[base_features]
y = train_data['Survived']

# Con nuevas características
X_new = train_data[new_features]

# Dividir en conjuntos de entrenamiento y prueba
X_train_base, X_test_base, y_train, y_test = train_test_split(X_base, y, test_size=0.3, random_state=42)
X_train_new, X_test_new, _, _ = train_test_split(X_new, y, test_size=0.3, random_state=42)

# Función para entrenar y evaluar modelos
def evaluar_modelo(model, X_train, X_test, y_train, y_test, desc):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"{desc} - Accuracy: {acc:.4f}, F1-Score: {f1:.4f}")
    return acc, f1

# Modelos a usar
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
tree_model = DecisionTreeClassifier(max_depth=5, random_state=42)

# Evaluación sin nuevas características
print("Evaluación sin nuevas características:")
evaluar_modelo(logistic_model, X_train_base, X_test_base, y_train, y_test, "Logistic Regression")
evaluar_modelo(tree_model, X_train_base, X_test_base, y_train, y_test, "Decision Tree")

# Evaluación con nuevas características
print("\nEvaluación con nuevas características:")
evaluar_modelo(logistic_model, X_train_new, X_test_new, y_train, y_test, "Logistic Regression")
evaluar_modelo(tree_model, X_train_new, X_test_new, y_train, y_test, "Decision Tree")


Evaluación sin nuevas características:
Logistic Regression - Accuracy: 0.8134, F1-Score: 0.7642
Decision Tree - Accuracy: 0.7948, F1-Score: 0.7442

Evaluación con nuevas características:
Logistic Regression - Accuracy: 0.7985, F1-Score: 0.7568
Decision Tree - Accuracy: 0.8209, F1-Score: 0.7757


(0.8208955223880597, 0.7757009345794392)